In [644]:
import pandas as pd
import sklearn

In [645]:
listing = pd.read_csv("listing_us.csv")

/var/folders/wq/lz1mv8gj227bv83rhxpf97rh0000gn/T/ipykernel_41759/835629374.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  listing = pd.read_csv("listing_us.csv")


In [646]:
listing.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'city'],
      dtype='object')

In [647]:
round(listing["room_type"].value_counts()/len(listing),4)*100

room_type
Entire home/apt    72.86
Private room       25.74
Shared room         0.98
Hotel room          0.42
Name: count, dtype: float64

In [648]:
round(listing[["price", "minimum_nights", "number_of_reviews", "reviews_per_month", "calculated_host_listings_count", "availability_365", "number_of_reviews_ltm"]].describe(),2)[1:]

,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
mean,259.47,13.50,40.92,1.64,29.88,180.99,11.69
std,1024.65,27.92,80.65,1.91,106.01,134.72,20.60
min,0.00,1.00,0.00,0.01,1.00,0.00,0.00
25%,91.00,2.00,1.00,0.31,1.00,52.00,0.00
50%,149.00,3.00,9.00,1.00,2.00,175.00,3.00
75%,250.00,30.00,43.00,2.42,10.00,321.00,16.00
max,100000.00,1250.00,3091.00,101.42,1003.00,365.00,1314.00


In [649]:
listing

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,city
0,958,"Bright, Modern Garden Unit - 1BR/1BTH",1169,Holly,NaN,Western Addition,37.770280,-122.433170,Entire home/apt,202,2,383,2023-02-19,2.31,1,128,59,San Francisco
1,5858,Creative Sanctuary,8904,Philip And Tania,NaN,Bernal Heights,37.744740,-122.420890,Entire home/apt,235,30,111,2017-08-06,0.66,1,365,0,San Francisco
2,8142,Friendly Room Apt. Style -UCSF/USF - San Franc...,21994,Aaron,NaN,Haight Ashbury,37.765550,-122.452130,Private room,56,32,9,2022-10-27,0.09,13,365,1,San Francisco
3,8339,Historic Alamo Square Victorian,24215,Rosy,NaN,Western Addition,37.775640,-122.436420,Entire home/apt,575,9,28,2019-06-28,0.17,2,365,0,San Francisco
4,8739,"Mission Sunshine, with Private Bath",7149,Ivan & Wendy,NaN,Mission,37.760300,-122.421970,Private room,110,1,770,2023-02-25,4.65,2,159,34,San Francisco
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232142,849230448624862502,Private Half Moon Bay Oasis By Pillar Point Be...,29395385,Khalid,NaN,Unincorporated Areas,37.506164,-122.476188,Entire home/apt,389,2,0,NaN,NaN,1,365,0,San Mateo County
232143,850805127216414630,Home near SFO,440957696,Christina,NaN,San Bruno,37.623630,-122.426570,Entire home/apt,158,1,0,NaN,NaN,1,364,0,San Mateo County
232144,851562104616413652,Modern New Private Studio in SFO,41574587,Vality,NaN,South San Francisco,37.655661,-122.413130,Entire home/apt,97,1,0,NaN,NaN,32,359,0,San Mateo County
232145,851792795339759410,Lux Bedroom with a Shared Bath near SFO,73133543,Stephannie,NaN,South San Francisco,37.644792,-122.454213,Private room,62,2,0,NaN,NaN,3,67,0,San Mateo County


In [650]:
#Create Category outcome by price threshold. I feel 50 bucks per night is good threshold. Over 50, i feel expensive.
def categorize_price(price):
    if price < 120:
        return 'low'
    else:
        return 'high'

In [651]:
listing['price_category'] = listing['price'].apply(categorize_price)

In [652]:
# create dummy variables for room type
dummies = pd.get_dummies(listing[["neighbourhood_group", "host_id"]], prefix="roomtype")
listing = pd.concat([listing, dummies], axis=1)

In [653]:
listing.drop(["room_type"], axis=1, inplace=True)

In [654]:
listing["price_category"].replace({"high":1, "low":0}, inplace=True)

/var/folders/wq/lz1mv8gj227bv83rhxpf97rh0000gn/T/ipykernel_41759/3717071175.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  listing["price_category"].replace({"high":1, "low":0}, inplace=True)
/var/folders/wq/lz1mv8gj227bv83rhxpf97rh0000gn/T/ipykernel_41759/3717071175.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silen

In [655]:
listing.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'price', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365',
       'number_of_reviews_ltm', 'city', 'price_category', 'host_id',
       'roomtype_Ballard', 'roomtype_Beacon Hill', 'roomtype_Bristol',
       'roomtype_Bronx', 'roomtype_Brooklyn', 'roomtype_Capitol Hill',
       'roomtype_Cascade', 'roomtype_Central Area',
       'roomtype_City of Los Angeles', 'roomtype_Delridge',
       'roomtype_Downtown', 'roomtype_Interbay', 'roomtype_Kent',
       'roomtype_Lake City', 'roomtype_Magnolia', 'roomtype_Manhattan',
       'roomtype_Newport', 'roomtype_Northgate', 'roomtype_Other Cities',
       'roomtype_Other neighborhoods', 'roomtype_Providence',
       'roomtype_Queen Anne', 'roomtype_Queens', 'roomtype_Rainier Valley',
       'roomtype_Seward Park', 'roomtype_Staten Island',
 

In [656]:
listing.drop(["id", "name", "host_id", "host_name", "neighbourhood","neighbourhood_group", "price", "last_review"], axis=1, inplace=True)

In [657]:
# Filter for city
listing_la = listing[listing["city"]=="Los Angeles"]
listing_ny = listing[listing["city"]=="New York City"]

listing_la.dropna(inplace=True)
listing_ny.dropna(inplace=True)

/var/folders/wq/lz1mv8gj227bv83rhxpf97rh0000gn/T/ipykernel_41759/4232467089.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listing_la.dropna(inplace=True)
/var/folders/wq/lz1mv8gj227bv83rhxpf97rh0000gn/T/ipykernel_41759/4232467089.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listing_ny.dropna(inplace=True)


In [658]:
X_la.columns

Index(['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'roomtype_Ballard',
       'roomtype_Beacon Hill', 'roomtype_Bristol', 'roomtype_Bronx',
       'roomtype_Brooklyn', 'roomtype_Capitol Hill', 'roomtype_Cascade',
       'roomtype_Central Area', 'roomtype_City of Los Angeles',
       'roomtype_Delridge', 'roomtype_Downtown', 'roomtype_Interbay',
       'roomtype_Kent', 'roomtype_Lake City', 'roomtype_Magnolia',
       'roomtype_Manhattan', 'roomtype_Newport', 'roomtype_Northgate',
       'roomtype_Other Cities', 'roomtype_Other neighborhoods',
       'roomtype_Providence', 'roomtype_Queen Anne', 'roomtype_Queens',
       'roomtype_Rainier Valley', 'roomtype_Seward Park',
       'roomtype_Staten Island', 'roomtype_Unincorporated Areas',
       'roomtype_University District', 'roomtype_Washington',
       'roomtype_West Seattle', 'roomtype_Entire home/apt',

In [659]:
X_la = listing_la.drop(["price_category","city"], axis=1).astype(int)
y_la = listing_la["price_category"]
X_ny = listing_ny.drop(["price_category", "city"], axis=1).astype(int)
y_ny = listing_ny["price_category"]

In [660]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [661]:
# Split data
X_la_train, X_la_test, y_la_train, y_la_test = train_test_split(X_la, y_la, test_size=0.3, random_state=42)
X_ny_train, X_ny_test, y_ny_train, y_ny_test = train_test_split(X_ny, y_ny, test_size=0.3, random_state=42)

## Logistic Regression

In [662]:
from sklearn.linear_model import LogisticRegression

In [663]:
lr_la = LogisticRegression()
lr_la.fit(X_la_train, y_la_train)
lr_ny = LogisticRegression()
lr_ny.fit(X_ny_train, y_ny_train)

/Users/moritakento/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/moritakento/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

LogisticRegression()

In [664]:
y_pred_la = lr_la.predict(X_la_test)
#y_pred_
y_pred_ny = lr_ny.predict(X_ny_test)

In [665]:
# model validation

accuracy_la = accuracy_score(y_la_test, y_pred_la)
conf_matrix_la = confusion_matrix(y_la_test, y_pred_la)
report_la = classification_report(y_la_test, y_pred_la)

print("Accuracy, Los Angeles:", accuracy_la)
print("\nConfusion Matrix, Los Angeles:\n", conf_matrix_la)
print("\nClassification Report, Los Angeles:\n", report_la)

Accuracy, Los Angeles: 0.6217240632445311

Confusion Matrix, Los Angeles:
 [[ 803 2949]
 [ 544 4938]]

Classification Report, Los Angeles:
               precision    recall  f1-score   support

           0       0.60      0.21      0.31      3752
           1       0.63      0.90      0.74      5482

    accuracy                           0.62      9234
   macro avg       0.61      0.56      0.53      9234
weighted avg       0.61      0.62      0.57      9234



In [666]:
# model validation
accuracy_ny = accuracy_score(y_ny_test, y_pred_ny)
conf_matrix_ny = confusion_matrix(y_ny_test, y_pred_ny)
report_ny = classification_report(y_ny_test, y_pred_ny)

print("Accuracy, New York:", accuracy_ny)
print("\nConfusion Matrix, New York:\n", conf_matrix_ny)
print("\nClassification Report, New York:\n", report_ny)

Accuracy, New York: 0.6205945448973338

Confusion Matrix, New York:
 [[2532 2128]
 [1586 3543]]

Classification Report, New York:
               precision    recall  f1-score   support

           0       0.61      0.54      0.58      4660
           1       0.62      0.69      0.66      5129

    accuracy                           0.62      9789
   macro avg       0.62      0.62      0.62      9789
weighted avg       0.62      0.62      0.62      9789



In [667]:
from sklearn.model_selection import cross_val_score

In [668]:
# クロスバリデーションを実行
cv_scores = cross_val_score(lr_la, X_la_train, y_la_train, cv=10, scoring='accuracy')

# クロスバリデーションのスコアを出力
print("Cross-Validation Scores: ", cv_scores)
print("Mean CV Score: ", cv_scores.mean())
print("Standard Deviation of CV Scores: ", cv_scores.std())

# トレーニング全体でモデルを学習し、テストデータを評価
log_reg.fit(X_la_train, y_la_train)
test_accuracy = accuracy_score(y_la_test, y_pred_la)
print("Test Accuracy: ", test_accuracy)

/Users/moritakento/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/moritakento/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Cross-Validation Scores:  [0.62320186 0.60603248 0.62320186 0.6287703  0.61206497 0.60835267
 0.62302693 0.6272052  0.61931291 0.63416899]
Mean CV Score:  0.6205338150357507
Standard Deviation of CV Scores:  0.008663708975360711
Test Accuracy:  0.6217240632445311


/Users/moritakento/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [669]:
# ロジスティック回帰モデルの初期化
log_reg = LogisticRegression(random_state=42, max_iter=1000)

# クロスバリデーションを実行
cv_scores_lr_ny = cross_val_score(log_reg, X_ny_train, y_ny_train, cv=10, scoring='accuracy')

# クロスバリデーションのスコアを出力
print("Cross-Validation Scores: ", cv_scores_lr_ny)
print("Mean CV Score: ", cv_scores_lr_ny.mean())
print("Standard Deviation of CV Scores: ", cv_scores_lr_ny.std())

# トレーニング全体でモデルを学習し、テストデータを評価
log_reg.fit(X_ny_train, y_ny_train)
test_accuracy_lr_ny = accuracy_score(y_ny_test, y_pred_ny)
print("Test Accuracy: ", test_accuracy_lr_ny)

/Users/moritakento/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/moritakento/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Cross-Validation Scores:  [0.60113835 0.61690018 0.64273205 0.61996497 0.62478109 0.62259194
 0.62521891 0.60113835 0.61191415 0.60534385]
Mean CV Score:  0.6171723843254757
Standard Deviation of CV Scores:  0.012226705874059935
Test Accuracy:  0.6205945448973338


## Decision Tree

In [670]:
from sklearn.tree import DecisionTreeClassifier

In [671]:
dt = DecisionTreeClassifier(criterion="entropy", random_state=42)
dt_la = dt.fit(X_la_train, y_la_train)
y_pred_dt_la = dt_la.predict(X_la_test)
dt_ny = dt.fit(X_ny_train, y_ny_train)
y_pred_dt_ny = dt_ny.predict(X_ny_test)

In [672]:
# model validation
accuracy_la_dt = accuracy_score(y_la_test, y_pred_dt_la)
conf_matrix_la_dt = confusion_matrix(y_la_test, y_pred_dt_la)
report_la_dt = classification_report(y_la_test, y_pred_dt_la)

print("Accuracy, Los Angeles:", accuracy_la_dt)
print("\nConfusion Matrix, Los Angeles:\n", conf_matrix_la_dt)
print("\nClassification Report, Los Angeles:\n", report_la_dt)

Accuracy, Los Angeles: 0.6509638293264024

Confusion Matrix, Los Angeles:
 [[2297 1455]
 [1768 3714]]

Classification Report, Los Angeles:
               precision    recall  f1-score   support

           0       0.57      0.61      0.59      3752
           1       0.72      0.68      0.70      5482

    accuracy                           0.65      9234
   macro avg       0.64      0.64      0.64      9234
weighted avg       0.66      0.65      0.65      9234



In [675]:
# model validation
accuracy_ny_dt = accuracy_score(y_ny_test, y_pred_dt_ny)
conf_matrix_ny_dt = confusion_matrix(y_ny_test, y_pred_dt_ny)
report_ny_dt = classification_report(y_ny_test, y_pred_dt_ny)

print("Accuracy, Los Angeles:", accuracy_ny_dt)
print("\nConfusion Matrix, Los Angeles:\n", conf_matrix_ny_dt)
print("\nClassification Report, Los Angeles:\n", report_ny_dt)

Accuracy, Los Angeles: 0.621207477781183

Confusion Matrix, Los Angeles:
 [[3019 1641]
 [2067 3062]]

Classification Report, Los Angeles:
               precision    recall  f1-score   support

           0       0.59      0.65      0.62      4660
           1       0.65      0.60      0.62      5129

    accuracy                           0.62      9789
   macro avg       0.62      0.62      0.62      9789
weighted avg       0.62      0.62      0.62      9789



In [673]:
# クロスバリデーションを実行
cv_scores_la_dt = cross_val_score(dt_la, X_la_train, y_la_train, cv=10, scoring='accuracy')

# クロスバリデーションのスコアを出力
print("Cross-Validation Scores: ", cv_scores_la_dt)
print("Mean CV Score: ", cv_scores_la_dt.mean())
print("Standard Deviation of CV Scores: ", cv_scores_la_dt.std())

# トレーニング全体でモデルを学習し、テストデータを評価
dt_la.fit(X_la_train, y_la_train)
test_accuracy_la_dt = accuracy_score(y_la_test, y_pred_dt_la)
print("Test Accuracy: ", test_accuracy_la_dt)

Cross-Validation Scores:  [0.63062645 0.65197216 0.64779582 0.65986079 0.65243619 0.64361949
 0.64948932 0.64856082 0.63974002 0.6536676 ]
Mean CV Score:  0.6477768657890032
Standard Deviation of CV Scores:  0.007743323615415981
Test Accuracy:  0.6509638293264024


In [674]:
# クロスバリデーションを実行
cv_scores_ny_dt = cross_val_score(dt_ny, X_ny_train, y_ny_train, cv=10, scoring='accuracy')

# クロスバリデーションのスコアを出力
print("Cross-Validation Scores: ", cv_scores_ny_dt)
print("Mean CV Score: ", cv_scores_ny_dt.mean())
print("Standard Deviation of CV Scores: ", cv_scores_ny_dt.std())

# トレーニング全体でモデルを学習し、テストデータを評価
dt_la.fit(X_ny_train, y_ny_train)
test_accuracy_ny_dt = accuracy_score(y_ny_test, y_pred_dt_ny)
print("Test Accuracy: ", test_accuracy_ny_dt)

Cross-Validation Scores:  [0.63397548 0.62434326 0.64492119 0.62171629 0.63222417 0.63003503
 0.63485114 0.6528021  0.63425318 0.62155059]
Mean CV Score:  0.6330672418461898
Standard Deviation of CV Scores:  0.00939619518976206
Test Accuracy:  0.621207477781183
